# Optimizing trayectories of Omnibot

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from numpy import logical_and as npand
from numpy import logical_or as npor

In [ ]:
from sympy import (symbols, pi, I, E, cos, sin, exp, tan, simplify, expand, factor, collect,
                   apart, cancel, expand_trig, diff, Derivative, Function, integrate, limit,
                   series, Eq, solve, dsolve, Matrix, N, preorder_traversal, Float, solve_linear_system,
                   eye, zeros, lambdify)
from sympy.physics.mechanics import dynamicsymbols, init_vprinting

In [ ]:
init_vprinting()

### Using the symbolic formulas to generate the numeric model

In [ ]:
def generic_omnibot_mats(n = 4, null_beta = True, equal_r = True):
    t, r, d, s = symbols('t r d s')
    alpha, beta= dynamicsymbols('alpha beta')
    
    W = Matrix([
        [r, cos(alpha)],
        [0, sin(alpha)]
    ])
    T = Matrix([
        [cos(beta), -sin(beta)],
        [sin(beta), cos(beta)]
    ])
    A = Matrix([
        [1, 0, -d],
        [0, 1, s]
    ])
    WTA= W.inv()@T.inv()@A
    WTA.simplify()
    r_n = WTA[0,:]
    s_n = WTA[1,:]
    
    R_list = []
    S_list = []
    
    for ii in range(n):
        r_ii = r_n.subs(alpha, symbols('alpha_'+str(ii+1)))
        r_ii = r_ii.subs(d, symbols('d_'+str(ii+1)))
        r_ii = r_ii.subs(s, symbols('s_'+str(ii+1)))
        s_ii = s_n.subs(alpha, symbols('alpha_'+str(ii+1)))
        s_ii = s_ii.subs(d, symbols('d_'+str(ii+1)))
        s_ii = s_ii.subs(s, symbols('s_'+str(ii+1)))
        if not equal_r:
            r_ii = r_ii.subs(r, symbols('r_'+str(ii+1)))
            s_ii = s_ii.subs(r, symbols('r_'+str(ii+1)))
        if null_beta:
            r_ii = r_ii.subs(beta, 0)
            s_ii = s_ii.subs(beta, 0)
        else:
            r_ii = r_ii.subs(beta, symbols('beta_'+str(ii+1)))
            s_ii = s_ii.subs(beta, symbols('beta_'+str(ii+1)))
            
        S_list.append(s_ii)
        R_list.append(r_ii)
    
    R = Matrix(R_list)
    S = Matrix(S_list)
    return R, S

In [ ]:
def integerize(expr):
    expr2 = expr
    for a in preorder_traversal(expr):
        if isinstance(a, Float):
            expr2 = expr2.subs(a, round(a))
    return expr2

def roundize(expr,n = 4):
    expr2 = expr
    for a in preorder_traversal(expr):
        if isinstance(a, Float):
            expr2 = expr2.subs(a, round(a,n))
    return expr2

In [ ]:
def dejabot_mats():
    R, S = generic_omnibot_mats()
    L, l = symbols('L l')
    for ii in range(4):
        alpha = pi/4 * (1 - 2 * ((int((ii+1)/2))%2))
        s = L * (1 - 2 * ((int((ii)/2))%2))
        d = l * (1 - 2 * (ii%2))
        R[ii,:] = R[ii,:].subs(symbols('alpha_'+str(ii+1)), alpha)
        R[ii,:] = R[ii,:].subs(symbols('s_'+str(ii+1)), s)
        R[ii,:] = R[ii,:].subs(symbols('d_'+str(ii+1)), d)
        S[ii,:] = S[ii,:].subs(symbols('alpha_'+str(ii+1)), alpha)
        S[ii,:] = S[ii,:].subs(symbols('s_'+str(ii+1)), s)
        S[ii,:] = S[ii,:].subs(symbols('d_'+str(ii+1)), d)
    return integerize(R), integerize(S)

In [ ]:
t, r, d, s, m, I_z, I_w, l, L = symbols('t r d s m I_z I_w l L')
x, y, alpha, beta, sigma, psi, theta= dynamicsymbols('x y alpha beta sigma psi theta')
psi_dot = psi.diff()
x, y, alpha, beta, sigma, psi, psi_dot, theta, t, r, d, s, m, I_z, I_w, l, L

In [ ]:
q = [x, y, psi] + [dynamicsymbols(f'phi_{i+1}') for i in range(4)]
q = Matrix(q)
Gamma = [dynamicsymbols(f'tau_{i+1}') for i in range(4)]
Gamma = Matrix(Gamma)

q, Gamma

In [ ]:
M_w = eye(4)*I_w
M_r = Matrix([
    [m, 0, 0],
    [0, m, 0],
    [0, 0, I_z]
])
R_psi = Matrix([
    [cos(psi), -sin(psi), 0],
    [sin(psi), cos(psi), 0],
    [0, 0, 1]
])
R, S = dejabot_mats()

M_r, M_w, R_psi, R

In [ ]:
H = M_r + R_psi@R.T@M_w@R@R_psi.T
H = simplify(H)
K = R_psi@R.T@M_w@R@R_psi.diff().T
K = simplify(K)
F_a = R_psi@R.T@Gamma
F_a_0 = F_a[0].factor(sin(psi), cos(psi))
F_a_1 = F_a[1].factor(sin(psi), cos(psi))
F_a_2 = F_a[2].factor()
F_a = Matrix([F_a_0, F_a_1, F_a_2])
A = R_psi@R.T
A_inv = R@R_psi.T
H_inv = simplify(H.inv())
H, K, F_a,

In [ ]:
get_h_raw = lambdify([m, I_w, I_z, l, L, r], H)
get_k_raw = lambdify([I_w, r, psi_dot], K)
get_a_raw = lambdify([psi, l, L, r], A)
get_a_inv_raw = lambdify([psi, l, L, r], A_inv)
get_h_inv_raw = lambdify([m, I_w, I_z, l, L, r], H_inv)

## Electric Motor Model

$$ V = K_m \dot{\phi} + Ri$$
$$ \tau_m = K_ei - \tau_r$$

In [ ]:
def friction(a, b, phi_dot, V, eps_phi = 0.1):
    fric_wet = a * phi_dot
    fric_dry = np.where(npor(
        phi_dot>eps_phi,
        npand(
            np.abs(phi_dot)<=eps_phi,
            V > 0)
    ),b, 0)
    fric_dry = np.where(npor(
        phi_dot< -eps_phi,
        npand(
            np.abs(phi_dot)<=eps_phi,
            V < 0)
    ),-b, fric_dry)
    return fric_wet + fric_dry

In [ ]:
def create_motor_model(i_7000 = 0.5):
    V = 24
    R = 24/13
    K_m = (V - R*i_7000)/(2 * np.pi*7000/60)
    t_rat = 4300*9.8/(100*1000)    #570*9.8/(100*1000)
    phi_rat = 0                    #5900*np.pi*2/60
    i_rat = (V - K_m*phi_rat)/R
    K_e = t_rat/(i_rat-i_7000)
    t_r = -t_rat + K_e * i_rat
    print(f'K_m = {K_m}, K_e = {K_e}, R = {R}, i_rat = {i_rat}, t_r = {t_r}')
    def motor_model(phi_dot, V = 24):
        i = (V - K_m*phi_dot)/R
        t = K_e * i - friction(0,t_r, phi_dot, V)
        p = t * phi_dot
        ef = p / (i * V)
        return t, i, p, ef
    def simp_motor_model(phi_dot, V = 24):
        i = (V - K_m*phi_dot)/R
        t = K_e * i - friction(0,t_r, phi_dot, V)
        return t
    return motor_model, simp_motor_model

In [ ]:
x = np.linspace(0, 2 * np.pi * 7000/60, 300)
n = x * 30 / np.pi
motor_model = create_motor_model(0.5)[0]
t, i, p, ef = motor_model(x)
t = t *100000/9.8

fig, ax1 = plt.subplots()

ax1.plot(t,n)
ax1.set_ylim([0,7000])
ax1.set_xlim([0,4400])

ax2 = ax1.twinx()
ax2.plot(t,i)
ax2.set_ylim([0,13])

ax3 = ax1.twinx()
ax3.plot(t,p)
ax3.spines["right"].set_position(("axes", 1.1))
ax3.set_ylim([0,80])

ax4 = ax1.twinx()
ax4.plot(t,ef)
ax4.spines["right"].set_position(("axes", 1.2))
ax4.set_ylim([0,1])
plt.grid()
plt.title('Motor Curves')

plt.grid(True)

### Comparison with manufacturer original motor curves:
![Original curves](images_dejabot/curvas_motor.png)

Expanded curves outside normal regime:

In [ ]:
x = np.linspace(-2 * np.pi * 7000/60, 4 * np.pi * 7000/60, 300)
n = x * 30 / np.pi
motor_model = create_motor_model(0.5)[0]
t, i, p, ef = motor_model(x)
t = t *100000/9.8

fig, ax1 = plt.subplots()

ax1.plot(t,n, 'r')
ax1.set_ylim([-14000,14000])
ax1.set_xlim([-4400,8800])

ax2 = ax1.twinx()
ax2.plot(t,i)
ax2.set_ylim([-26,26])

ax3 = ax1.twinx()
ax3.plot(t,p)
ax3.spines["right"].set_position(("axes", 1.1))
ax3.set_ylim([-80,80])

ax4 = ax1.twinx()
ax4.plot(t,ef, 'g')
ax4.spines["right"].set_position(("axes", 1.2))
ax4.set_ylim([-2,2])
plt.grid()

plt.grid(True)

Adapted Curves with reductor gears:

In [ ]:
def create_motoreduct_model(i_7000 = 0.5, N = 49, eff = 0.6, a = 0.01, b = 0.4):
    V = 24
    R = 24/13
    K_m = (V - R*i_7000)/(2 * np.pi*7000/60)
    t_rat = 4300*9.8/(100*1000)    #570*9.8/(100*1000)
    phi_rat = 0                    #5900*np.pi*2/60
    i_rat = (V - K_m*phi_rat)/R
    K_e = t_rat/(i_rat-i_7000)
    
    print(f'K_m = {K_m}, K_e = {K_e}, R = {R}, i_rat = {i_rat}')
    eps_phi = 0.1
    
    def motoreduct_model(phi_dot, V = 24):
        phi_dot_motor = phi_dot * N
        i = (V - K_m*phi_dot_motor)/R
        t = K_e * i * N * eff - friction(a, b, phi_dot, V, eps_phi)
        p = t * phi_dot
        ef = p / (i * V)
        return t, i, p, ef
    def simp_motoreduct_model(phi_dot, V = 24):
        phi_dot_motor = phi_dot * N
        i = (V - K_m*phi_dot_motor)/R
        t = K_e * i * N * eff - friction(a, b, phi_dot, V, eps_phi)
        return t
    return motoreduct_model, simp_motoreduct_model

In [ ]:
x = np.linspace(0, 2 * np.pi * 7000/(49*60), 300)
n = x * 30 / np.pi
motor_model = create_motoreduct_model(0.5)[0]
t, i, p, ef = motor_model(x)
t = t *100000/9.8

fig, ax1 = plt.subplots()

ax1.plot(t,n)
ax1.set_ylim([0,150])
ax1.set_xlim([0,130000])

ax2 = ax1.twinx()
ax2.plot(t,i)
ax2.set_ylim([0,13])

ax3 = ax1.twinx()
ax3.plot(t,p)
ax3.spines["right"].set_position(("axes", 1.1))
ax3.set_ylim([0,80])

ax4 = ax1.twinx()
ax4.plot(t,ef)
ax4.spines["right"].set_position(("axes", 1.2))
ax4.set_ylim([0,1])
#plt.grid()

ax5 = ax1.twinx()
ax5.spines["right"].set_position(("axes", -0.2))
ax5.set_ylim([0, x[-1]*0.0667])

plt.grid(True)

In [ ]:
def torq_lims(phi_dot, v_max = 24, motor = create_motoreduct_model()[1]):
    t_max = motor(phi_dot, v_max)
    t_min = motor(phi_dot, -v_max)
    
    return t_max, t_min

In [ ]:
def vect_torq_lims(geom, q, q_dot):
    a = get_a(geom, q)
    phi_dot = a.T@q_dot
    #print(f'phi_dot: {np.round(phi_dot,3)}')
    t_max = []
    t_min = []
    for wheel in phi_dot:
        maxt, mint = torq_lims(wheel)
        t_max.append(maxt)
        t_min.append(mint)
    t_max = np.array(t_max)
    t_min = np.array(t_min)
    
    return (t_max, t_min)

## Parameter encoding in functions:
$$Geom = [l, L, r]$$
$$Mass = [m, I_w,I_z]$$
$$q = [x, y, \psi]$$ where q is $q_r$

In [ ]:
def get_h(geom, mass):
    l, L, r = geom
    m, I_w, I_z = mass
    return get_h_raw(m, I_w, I_z, l, L, r)

def get_k(geom, mass, q_dot):
    l, L, r = geom
    m, I_w, I_z = mass
    x_dot, y_dot, psi_dot = q_dot
    return get_k_raw(I_w, r, psi_dot)

def get_a(geom, q):
    l, L, r = geom
    x, y, psi = q
    return get_a_raw(psi, l, L, r)

def get_h_inv(geom, mass):
    l, L, r = geom
    m, I_w, I_z = mass
    return get_h_inv_raw(m, I_w, I_z, l, L, r)


In [ ]:
geom = [0.2096, 0.2096, 0.0667]
mass = [15.75, 0.00266, 0.461]
q_0 = np.array([0, 0, 0])
q_dot_0 = np.array([0, 0, 0])
vect_torq_lims(geom, q_0, q_dot_0+[0.9,0,0])

# Casadi functions

In [ ]:
import casadi as cas

In [ ]:
a = cas.SX.sym('a')
b = cas.SX.sym('b')
#phi_dot = cas.SX.sym('phi_dot', 4)
V = cas.SX.sym('V', 4)
geom = [0.2096, 0.2096, 0.0667]
mass = [15.75, 0.00266, 0.461]
q_cas = cas.SX.sym('q', 3)
q_dot_cas = cas.SX.sym('q_dot', 3)
q_dot_dot_cas = cas.SX.sym('q_dot_dot', 3)

In [ ]:
def get_k_cas(geom, mass, q_dot):
    l, L, r = geom
    m, I_w, I_z = mass
    x_dot, y_dot, psi_dot = cas.vertsplit(q_dot)
    return cas.SX(get_k_raw(I_w, r, psi_dot))

def get_a_cas(geom, q):
    l, L, r = geom
    x, y, psi = cas.vertsplit(q)
    return cas.SX(get_a_raw(psi, l, L, r))

def get_a_inv_cas(geom, q):
    l, L, r = geom
    x, y, psi = cas.vertsplit(q)
    return cas.SX(get_a_inv_raw(psi, l, L, r))


In [ ]:
h_cas = get_h(geom, mass)
h_inv_cas = get_h_inv(geom, mass)
k_cas = get_k_cas(geom, mass, q_dot_cas)
k_cas_fun = cas.Function('K', [q_dot_cas], [k_cas])
a_cas = get_a_cas(geom, q_cas)
a_cas_fun = cas.Function('A', [q_cas], [a_cas])
a_inv_cas = get_a_inv_cas(geom, q_cas)
a_inv_cas_fun = cas.Function('A_inv', [q_cas], [a_inv_cas])

In [ ]:
phi_dot = a_inv_cas @ q_dot_cas
phi_dot

In [ ]:
def fric_cas(a, b, phi_dot):
    fric_wet = a * phi_dot
    fric_dry = b * cas.sign(phi_dot)
    return fric_wet + fric_dry

In [ ]:
fric_cas(a, b, phi_dot)

In [ ]:
def create_motoreduct_model_cas(i_7000 = 0.5, N = 49, eff = 0.6, a = 0.01, b = 0.4):
    V = 24
    R = 24/13
    K_m = (V - R*i_7000)/(2 * np.pi*7000/60)
    t_rat = 4300*9.8/(100*1000)    #570*9.8/(100*1000)
    phi_rat = 0                    #5900*np.pi*2/60
    i_rat = (V - K_m*phi_rat)/R
    K_e = t_rat/(i_rat-i_7000)
    
    print(f'K_m = {K_m}, K_e = {K_e}, R = {R}, i_rat = {i_rat}')
    eps_phi = 0.1
    
    def motoreduct_model(phi_dot, V = 24):
        phi_dot_motor = phi_dot * N
        i = (V - K_m*phi_dot_motor)/R
        t = K_e * i * N * eff - fric_cas(a, b, phi_dot)
        p = t * phi_dot
        ef = p / (i * V)
        return t, i, p, ef
    def simp_motoreduct_model(phi_dot, V = 24):
        phi_dot_motor = phi_dot * N
        i = (V - K_m*phi_dot_motor)/R
        t = K_e * i * N * eff - fric_cas(a, b, phi_dot)
        return t
    return motoreduct_model, simp_motoreduct_model

In [ ]:
motored_cas, simp_motored_cas = create_motoreduct_model_cas()

In [ ]:
t_cas, i_cas, p_cas, ef_cas = motored_cas(phi_dot, V)
t_cas

In [ ]:
h_cas, k_cas, a_cas

In [ ]:
h_cas @ q_dot_dot_cas

In [ ]:
k_cas @ q_dot_cas

In [ ]:
a_cas @ t_cas

In [ ]:
eq_total = h_inv_cas @ (-k_cas @ q_dot_cas + a_cas @ t_cas)
eq_total

In [ ]:
eq_tot_fun = cas.Function('eq_tot_fun', [q_cas, q_dot_cas, V], [eq_total])

In [ ]:
eq_tot_fun([1,1,1],[1,1,1], [1,1,1,1])

In [ ]:
tor_cas_eq = cas.Function('tau_cas', [q_cas, q_dot_cas, V], [a_cas @ t_cas])

In [ ]:
tor_cas_eq([0,0,0], [0.993529,0,0], [24,24,24,24])

In [ ]:
pot_cas = V.T @ i_cas
pot_cas

In [ ]:
pot_cas_eq = cas.Function('pot_cas', [q_cas, q_dot_cas, V], [pot_cas])

### Esquema de integración

In [ ]:
t = cas.SX.sym('t')
dt = cas.SX.sym('dt')
u = cas.SX.sym('u', 4)

#rhs = vertcat(x[1], u-sin(x[0]))
#rhs = vertcat(x[1], u)
#F = Function('F', [x, u], [rhs])

#k1 = F(x, u);
#k2 = F(x + dt/2 * k1, u)
#k3 = F(x + dt/2 * k2, u)
#k4 = F(x + dt * k3, u);
#new_x_expr = x+dt/6*(k1 +2*k2 +2*k3 +k4)

#new_x_expr = x + dt * F(x, u)
new_q_expr = q_cas + dt * q_dot_cas + 0.5 * q_dot_dot_cas * dt**2
new_q = cas.Function('New_q', [q_cas, q_dot_cas, q_dot_dot_cas, dt], [new_q_expr])

new_q_dot_expr = q_dot_cas + q_dot_dot_cas * dt
new_q_dot = cas.Function('New_q_dot', [q_dot_cas, q_dot_dot_cas, dt], [new_q_dot_expr])

### Objetivo

In [ ]:
q_f = cas.SX.sym('q_f', 3)
pos_diff = q_cas - q_f

dist = pos_diff.T @ pos_diff
tot_ene_cas = cas.SX.sym('tot_ene')
cost_expr = dist + tot_ene_cas
cost_fun = cas.Function('cost_fun', [q_cas, q_f, tot_ene_cas], [cost_expr])

In [ ]:
cost_fun([0,0,0], [3,4,0], [0])

## Opti problem

In [ ]:
N = 20

opti = cas.Opti()
opti.solver('ipopt')

Q = opti.variable(N+1,3)
Q_dot = opti.variable(N+1,3)
Q_dot_dot = opti.variable(N,3)
U = opti.variable(N, 4)
#T = opti.variable()
dt = opti.parameter()
u_m = opti.parameter()
Q_f = opti.parameter(3)
#t_m = opti.parameter()

cost = cost_fun(Q[-1,:], Q_f, 0)
opti.minimize(cost)

opti.subject_to(Q[0,:].T == [0, 0, 0])
opti.subject_to(Q_dot[0,:].T == [0, 0, 0])


for ii in range(N):
    opti.subject_to(Q_dot_dot[ii,:] == eq_tot_fun(Q[ii,:], Q_dot[ii,:], U[ii,:]).T)
    opti.subject_to(Q[ii+1,:].T == new_q(Q[ii,:], Q_dot[ii,:], Q_dot_dot[ii,:], dt))
    opti.subject_to(Q_dot[ii+1,:].T == new_q_dot(Q_dot[ii,:], Q_dot_dot[ii,:], dt))
    for jj in range(4):
        opti.subject_to(opti.bounded(-u_m,U[ii,jj],u_m))

In [ ]:
N = 20

opti = cas.Opti()
opti.solver('ipopt')

#Q = opti.variable(N+1,3)
#Q_dot = opti.variable(N+1,3)
#Q_dot_dot = opti.variable(N,3)
U = opti.variable(N, 4)
dt = opti.parameter()
u_m = opti.parameter()
Q_f = opti.parameter(3)

tot_ene = 0


Q = np.array([[0, 0, 0]])
Q_dot = np.array([[0, 0, 0]])


for ii in range(N):
    tot_ene = tot_ene + pot_cas_eq(Q, Q_dot, U[ii,:])*dt
    Q_dot_dot = eq_tot_fun(Q, Q_dot, U[ii,:]).T
    Q = new_q(Q, Q_dot, Q_dot_dot, dt).T
    Q_dot = new_q_dot(Q_dot, Q_dot_dot, dt).T
    for jj in range(4):
        opti.subject_to(opti.bounded(-u_m,U[ii,jj],u_m))


cost = cost_fun(Q, Q_f,tot_ene*0)
opti.minimize(cost)


In [ ]:
u_ej = np.ones([N,4])
u_ej[:,0] = 0
u_ej[:,-1] = 0
opti.set_initial(U, u_ej)
#opti.set_initial(X[:,1], pi/N)
#opti.set_initial(T, 50)
opti.set_value(dt, 0.001)
max_volt = 24
opti.set_value(u_m, max_volt)
opti.set_value(Q_f, [1,1,0])

In [ ]:
sol = opti.solve()

## Casadi Results

In [ ]:
def integrar_step_euler(q, q_dot, u, dt):
    q_dot_dot = h_inv_cas @ (tor_cas_eq(q, q_dot, u)  - k_cas_fun(q_dot)@ q_dot)
    q_new = new_q(q, q_dot, q_dot_dot, dt)
    q_dot_new = new_q_dot(q_dot, q_dot_dot, dt)
    return np.array(q_new).flatten(), np.array(q_dot_new).flatten()

In [ ]:
def integrar_step_euler_func(q, q_dot, u, dt):
    q_dot_dot = eq_tot_fun(q, q_dot, u)
    q_new = new_q(q, q_dot, q_dot_dot, dt)
    q_dot_new = new_q_dot(q_dot, q_dot_dot, dt)
    return np.array(q_new).flatten(), np.array(q_dot_new).flatten()

In [ ]:
def integrar_intervalo_euler(q_0, q_dot_0, u, N, dt):
    q = np.zeros([N+1, 3])
    q_dot = np.zeros([N+1, 3])
    q[0,:] = q_0
    q_dot[0,:] = q_dot_0
    for ii in range(N):
        q[ii+1, :], q_dot[ii+1, :] = integrar_step_euler(q[ii,:], q_dot[ii,:], u[ii,:], dt)
    return q, q_dot

In [ ]:
Q_result = sol.value(Q)
U_result = sol.value(U)
dt_used = sol.value(dt)
qf_used = sol.value(Q_f)


q_integ, q_dot_integ = integrar_intervalo_euler([0,0,0], [0,0,0], U_result, N, dt_used)
Result = U_result, q_integ, dt_used, qf_used 

In [ ]:
def cost_fun_array(q, qf):
    N = q.shape[0]
    diff = np.zeros(N)
    for ii in range(N):
        diff[ii] = cost_fun(q[ii,:], qf)
    return diff

In [ ]:
plt.figure(figsize=[10,8])
q_integ, q_dot_integ = integrar_intervalo_euler([0,0,0], [0,0,0], U_result, N, dt_used)
plt.plot(q_integ[:,0], label = 'x')
plt.plot(q_integ[:,1], label = 'y')
plt.plot(q_integ[:,2], label = '$\psi$')
for ii in range(3):
    plt.scatter(N, Q_result[ii])

plt.legend()

In [ ]:
plt.figure(figsize=[10,10])
u_ej = np.ones([N,4])*24
u_ej[:,0] = 0
u_ej[:,-1] = 0
q_integ_2, q_dot_integ_2 = integrar_intervalo_euler([0,0,0], [0,0,0], u_ej, N, dt_used)
diff_arr = cost_fun_array(q_integ, qf_used)
diff_arr_2 = cost_fun_array(q_integ_2, qf_used)

plt.plot(diff_arr_2, label = 'cost diagonal')
plt.plot(q_integ_2[:,0], label = 'x')
plt.plot(q_integ_2[:,1], label = 'y')
plt.plot(q_integ_2[:,2], label = '$\psi$')
plt.plot(diff_arr, ':', label = 'cost casadi')
plt.plot(q_integ[:,0], ':', label = 'x')
plt.plot(q_integ[:,1], ':', label = 'y')
plt.plot(q_integ[:,2], ':', label = '$\psi$')
plt.legend()

In [ ]:
plt.plot(U_result[:,0], label = "V_1")
plt.plot(U_result[:,1], label = "V_2")
plt.plot(U_result[:,2], label = "V_3")
plt.plot(U_result[:,3], label = "V_4")
plt.legend()

In [ ]:
import datetime as datetime

In [ ]:
def timer_dec(f):
    def new_f(*args, **kwargs):
        pre_time = datetime.datetime.now()
        results = f(*args, **kwargs)
        post_time = datetime.datetime.now()
        time_delta = post_time - pre_time
        return results, time_delta.total_seconds()
    return new_f

In [ ]:
@timer_dec
def foo(x):
    for ii in range(100):
        print('hola')
    return 'hoho'

In [ ]:
@timer_dec
def casadi_omnibot(N = 20, q_f = [1,1,0], q_i = np.array([[0, 0, 0]]),
                   q_dot_i = np.array([[0, 0, 0]]), d_t = 0.01):

    opti = cas.Opti()
    opti.solver('ipopt')

    U = opti.variable(N, 4)
    dt = opti.parameter()
    u_m = opti.parameter()
    Q_f = opti.parameter(3)


    Q = q_i
    Q_dot = q_dot_i


    for ii in range(N):
        Q_dot_dot = eq_tot_fun(Q, Q_dot, U[ii,:]).T
        Q = new_q(Q, Q_dot, Q_dot_dot, dt).T
        Q_dot = new_q_dot(Q_dot, Q_dot_dot, dt).T
        for jj in range(4):
            opti.subject_to(opti.bounded(-u_m,U[ii,jj],u_m))


    cost = cost_fun(Q, Q_f,0)
    opti.minimize(cost)
    
    u_ej = np.ones([N,4])
    u_ej[:,0] = 0
    u_ej[:,-1] = 0
    opti.set_initial(U, u_ej)
    opti.set_value(dt, d_t)
    max_volt = 24
    opti.set_value(u_m, max_volt)
    opti.set_value(Q_f, q_f)
    
    sol = opti.solve()
    
    Q_result = sol.value(Q)
    U_result = sol.value(U)
    dt_used = sol.value(dt)
    qf_used = sol.value(Q_f)
    
    return U_result, Q_result, dt_used, qf_used

In [ ]:
N_values = [5, 10, 20, 30, 40, 50, 75, 100]
Time_values = []
results = []
for ii in N_values:
    new_r, new_t = casadi_omnibot(N = ii)
    Time_values.append(new_t)
    results.append(new_r)

In [ ]:
plt.figure(figsize=[12,8])
plt.plot(N_values, Time_values)
plt.grid()

In [ ]:
for ii, jj in zip(N_values, Time_values):
    print('N:', ii, '\tsimulated time:', ii*0.01, '\tseconds:',jj)

In [ ]:
def calc_dist(q1, q2):
    dist = q1-q2
    dist = dist @ dist.T
    return dist ** 0.5

In [ ]:
def long_path_casadi_omnibot(N = 20, q_f = [1,1,0], q_i = np.array([[0, 0, 0]]),
                   q_dot_i = np.array([[0, 0, 0]]), d_t = 0.01):
    epsilon = 0.001
    tray = q_i
    tray_vel = q_dot_i
    for iter_count in range(20):
        dist = calc_dist(q_i, q_f)
        if dist < epsilon:
            break
        print('\nIteration', iter_count, 'q_i = ', q_i, '\n')
        new_r, new_t = casadi_omnibot(N, q_f, q_i, q_dot_i, d_t)
        U_result, Q_result, dt_used, qf_used = new_r
        q_integ, q_dot_integ = integrar_intervalo_euler(q_i, q_dot_i, U_result, N, dt_used)
        tray = np.concatenate((tray, q_integ[:N//2,:]))
        tray_vel = np.concatenate((tray_vel, q_dot_integ[:N//2,:]))
        q_i = tray[-1,:]
        q_dot_i = tray_vel[-1,:]
    return tray, tray_vel

In [ ]:
tray, tray_vel = long_path_casadi_omnibot()

In [ ]:
Result = U_result, tray, dt_used, qf_used 

In [ ]:
tray

In [ ]:
from matplotlib import animation, rc
import matplotlib.patches as patches
import matplotlib as mpl

from IPython.display import HTML

In [ ]:
def add_omnibot_anim(ax, update_list, col, L = 0.2096):
    L = 0.2096
    tri_coord = np.array([
        [L/2,0],
        [L/4, L/4],
        [L/4, -L/4]
    ])
    
    omnibox = []
    for ii,jj in zip([1,-1,1,-1], [1,1,-1,-1]):
        whe = patches.Rectangle((ii*L/4, jj*L/2), ii*L*0.2, jj*L*0.1,
                                  linewidth=1,edgecolor='k',facecolor= col, alpha = 0.8)
        wheel = ax.add_patch(whe)
        omnibox.append(wheel)
        update_list.append(wheel)
    rect = patches.Rectangle((-L/2, -L/2), L, L, linewidth=1,edgecolor='k',facecolor= col, alpha = 0.3)
    tri = plt.Polygon(tri_coord, color= col)
    
    # Add the patch to the Axes
    omnibox.append(ax.add_patch(rect))
    omnibox.append(ax.add_patch(tri))
    update_list.append(ax.add_patch(rect))
    update_list.append(ax.add_patch(tri))
    
    return omnibox

In [ ]:
def transf_omnibot(omnibox, q, i, ax):
    x,y,theta = q[i,:]
    t1 = mpl.transforms.Affine2D().translate(x,y)
    t2 = mpl.transforms.Affine2D().rotate_around(x,y,theta) + ax.transData
    for element in omnibox:
        element.set_transform(t1+t2)

In [ ]:
def create_anim(result):
    u, q, dt, qf = result
    N = q.shape[0]
    
    fig, ax = plt.subplots()

    fig.set_size_inches([8,8])
    ax.set_xlim(( -1, 1))
    ax.set_ylim(( -1, 1))
    plt.grid()

    update_list = []
    omnibox = add_omnibot_anim(ax, update_list, 'g')
    def init():
        return update_list
    def animate(i):
        transf_omnibot(omnibox, q, i, ax)
        return update_list
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=N, interval=20, 
                               blit=True)
    return anim

In [ ]:
anim = create_anim(Result)

In [ ]:
HTML(anim.to_jshtml())

## Bot class creation

In [ ]:
class Bot:
    def __init__(self, geom, mass, q_init, q_dot_init, u_func):
        self.geom = geom
        self.mass = mass
        self.q = q_init
        self.q_dot = q_dot_init
        self.get_u = u_func
        self.t = 0
        self.H = get_h(geom, mass)
        self.H_inv = get_h_inv(geom, mass)
        
    def get_a(self):
        return get_a(self.geom, self.q)
    
    def get_k(self):
        return get_k(self.geom, self.mass, self.q_dot)
    
    def vect_torq_lims(self):
        return vect_torq_lims(self.geom, self.q, self.q_dot)
    
    def max_q_sec_dir(self, alpha, beta, print_status = False):
        t_max, t_min = self.vect_torq_lims()
        A = self.get_a()
        B = np.linalg.pinv(A)
        q_sec = np.array([np.cos(alpha)*np.cos(beta), np.sin(alpha)*np.cos(beta),np.sin(beta)])
        v_1 = B@self.H@q_sec
        v_2 = B@self.get_k()@self.q_dot
        a_max = []
        for ii in range(4):
            a_1 = (t_max[ii]-v_2[ii])/v_1[ii]
            a_2 = (t_min[ii]-v_2[ii])/v_1[ii]
            a_max.append(max(a_1, a_2))
        if print_status:
            print(f'alpha = {np.round(alpha,3)}, \n\tv1 = {np.round(v_1,3)} \n\ta_max = {np.round(a_max,3)}')
        return min(a_max)
        
    
    def q_sec_lims(self):
        torq_lims = self.vect_torq_lims()
        q_sec_max = self.H_inv@(self.get_a()@torq_lims[0]-self.get_k()@self.q_dot)
        q_sec_min = self.H_inv@(self.get_a()@torq_lims[1]-self.get_k()@self.q_dot)
        return q_sec_max, q_sec_min
    
    def step(self, d_t, envir = None):
        u = self.get_u(self, d_t, envir)
        self.q = self.q + d_t * self.q_dot + 0.5 * u * d_t**2
        self.q_dot = self.q_dot + d_t * u
        self.t = self.t + d_t

In [ ]:
geom = [0.2096, 0.2096, 0.0667]
mass = [15.75, 0.00266, 0.461]
q_0 = np.array([0, 0, 0])
q_dot_0 = np.array([0, 0, 0])
def dummy_u(bot, dt, envir):
    if envir:
        return(np.array(envir))
    #print(f't = {round(t,3)}, q = {q}, q_dot = {q_dot}')
    return(np.array([0,1,0]))
dejabot = Bot(geom, mass, q_0, q_dot_0, dummy_u)

## Bot class characteristics and capacities

In [ ]:
#dejabot.step(0.1)
print(f'q_dot: {dejabot.q_dot}\nq: {dejabot.q}')
x = np.linspace(0,1*2*np.pi, 401)
y = []
for ii in x:
    y.append(dejabot.max_q_sec_dir(ii, 0, False)/41.3)
y = np.array(y)
psi = dejabot.q[2]
w,h = plt.figaspect(1)
plt.figure(figsize = [w,h])
plt.plot(y*np.cos(x), y*np.sin(x), 'r')
plt.plot(y*np.cos(x-psi), y*np.sin(x-psi), 'b:')
plt.grid()

In [ ]:
%matplotlib notebook

In [ ]:
fig = plt.figure()
ax = plt.axes(projection="3d")

alpha = np.linspace(0,2*np.pi, 50)
beta = np.linspace(-np.pi,np.pi, 50)
dist = []
for aa in alpha:
    for bb in beta:
        dist.append([dejabot.max_q_sec_dir(aa, bb, False), aa,bb])
dist = np.array(dist)

z_points = dist[:,0] * np.sin(dist[:,2])
x_points = dist[:,0] * np.cos(dist[:,1]) * np.cos(dist[:,2])
y_points = dist[:,0] * np.sin(dist[:,1]) * np.cos(dist[:,2])
ax.scatter3D(x_points, y_points, z_points, s = 10);


In [ ]:
%matplotlib inline

In [ ]:
for dt in [0.01, 0.02, 0.03, 0.04, 0.05]:
    dejabot = Bot(geom, mass, q_0, q_dot_0, dummy_u)
    q_acum = [dejabot.q_dot[0],]
    for i in np.arange(0, 0.5+dt, dt):
        F = dejabot.max_q_sec_dir(np.pi/2, 0, False)
        dejabot.step(dt,[F,0,0])
        q_acum.append(dejabot.q_dot[0])
    q_acum = np.array(q_acum)
    plt.plot(np.arange(0, 0.5+ 2 *dt, dt), q_acum, label = 'dt = '+str(dt))
    #print(f'F: {F}\nq_dot: {dejabot.q_dot}\nq: {dejabot.q}')
plt.legend()
plt.grid()

In [ ]:
def calc_alpha(q, goal):
    if abs(goal[0]-q[0])==0:
        alpha = np.pi/2
    else:
        alpha = np.arctan((goal[1]-q[1])/(goal[0]-q[0]))
    if (goal[0]-q[0]) < 0:
        alpha = alpha + np.pi
    return alpha

## Gradient Descent approach

In [ ]:
goal = [0,1,0]
obstacles = []
envir = [goal, obstacles]
def simple_grad_desc(bot, dt, envir):
    q = bot.q
    goal, obstacles = envir
    alpha = calc_alpha(q, goal)
    a = bot.max_q_sec_dir(alpha, 0, False)
    u = [a*np.cos(alpha), a*np.sin(alpha),0]
    
    #print(f'alpha = {round(alpha,3)}, goal-q = {np.round(np.array(goal)-q,4)}, u = {np.round(u,4)}')
    
    return(np.array(u))

In [ ]:
goal = [-1,0,0]
obstacles = []
envir = [goal, obstacles]
dt = 0.01
n = 450
dejabot = Bot(geom, mass, q_0, q_dot_0, simple_grad_desc)
x_acum = [dejabot.q[0],]
x_dot_acum = [dejabot.q[0],]
y_acum = [dejabot.q[1],]
u_hist = [dejabot.get_u(dejabot, dt, envir)]
alpha_hist = [calc_alpha(dejabot.q, goal)]
for i in range(n):
    dejabot.step(dt,envir)
    x_acum.append(dejabot.q[0])
    y_acum.append(dejabot.q[1])
    x_dot_acum.append(dejabot.q_dot[0])
    u_hist.append(dejabot.get_u(dejabot, dt, envir))
    alpha_hist.append(calc_alpha(dejabot.q, goal))
x_acum = np.array(x_acum)
y_acum = np.array(y_acum)
x_dot_acum = np.array(x_dot_acum)
u_hist = np.array(u_hist)
alpha_hist = np.array(alpha_hist)
plt.plot(np.arange(n+1), x_acum, label = 'x')
#plt.plot(np.arange(n+1), y_acum, label = 'y')
plt.plot(np.arange(n+1), x_dot_acum, label = 'x_dot')
plt.plot(np.arange(n+1), u_hist[:,0]*0.01, label = 'u_x')
plt.plot(np.arange(n+1), alpha_hist/np.pi, label = 'alpha')
plt.legend()
plt.grid()
#print(f'F: {F}\nq_dot: {dejabot.q_dot}\nq: {dejabot.q}')

In [ ]:
plt.plot(x_acum, y_acum)

In [ ]:
def max_speed_dir(bot, alpha, max_motor_rpm = 7000, N = 49):
    A = bot.get_a()
    v = np.array([np.cos(alpha), np.sin(alpha),0])
    D = A.T@v
    b = []
    for ii in range(4):
        b_m = max_motor_rpm/(30*49*D[ii])
        b.append(abs(b_m))
    return min(b)

In [ ]:
geom = [0.2096, 0.2096, 0.0667]
mass = [15.75, 0.00266, 0.461]
q_0 = np.array([0, 0, 0])
q_dot_0 = np.array([0, 0, 0])
def dummy_u(bot, dt, envir):
    if envir:
        return(np.array(envir))
    #print(f't = {round(t,3)}, q = {q}, q_dot = {q_dot}')
    return(np.array([0,1,0]))
dejabot = Bot(geom, mass, q_0, q_dot_0, dummy_u)

print(f'q_dot: {dejabot.q_dot}\nq: {dejabot.q}')
x = np.linspace(0,2*np.pi, 401)
y = []
for ii in x:
    y.append(max_speed_dir(dejabot, ii))
y = np.array(y)
psi = dejabot.q[2]
w,h = plt.figaspect(1)
plt.figure(figsize = [w,h])
plt.plot(y*np.cos(x), y*np.sin(x), 'r')
plt.plot(y*np.cos(x+psi), y*np.sin(x+psi), 'b:')
plt.grid()

In [ ]:
goal = [0,1,0]
obstacles = []
envir = [goal, obstacles]
def better_grad_desc(bot, dt, envir, print_m = False):
    q = bot.q
    goal, obstacles = envir
    alpha = calc_alpha(q, goal)
    v_unit = np.array([np.cos(alpha), np.sin(alpha),0])
    max_speed = max_speed_dir(bot, alpha)
    dist = np.linalg.norm(np.matrix(goal)-q)
    
    one_step_speed = dist/dt
    speed = min(max_speed,  one_step_speed)
    q_dot = bot.q_dot
    delta_v = speed*v_unit - q_dot
    
    beta = calc_alpha([0,0,0], delta_v)
    max_acc = bot.max_q_sec_dir(beta, 0, False)
    one_step_accel = np.linalg.norm(delta_v)/dt
    accel = min(max_acc, one_step_accel)
    
    u = [accel*np.cos(beta), accel*np.sin(beta),0]
    if print_m:
        print(f'delta_v = {np.round(delta_v,4)}, beta = {np.round(beta*180/np.pi,4)}, a = {np.round(max_acc,4)} u = {np.round(u,4)}')
    
    return(np.array(u))

In [ ]:
goal = [1,1,0]
obstacles = []
envir = [goal, obstacles]
dt = 0.005
n = 450
q_0 = np.array([0, 0, 0])
q_dot_0 = np.array([-0.1, -0.2, 0])
dejabot = Bot(geom, mass, q_0, q_dot_0, better_grad_desc)
x_acum = [dejabot.q[0],]
x_dot_acum = [dejabot.q[0],]
y_acum = [dejabot.q[1],]
u_hist = [dejabot.get_u(dejabot, dt, envir)]
alpha_hist = [calc_alpha(dejabot.q, goal)]
for i in range(n):
    dejabot.step(dt,envir)
    x_acum.append(dejabot.q[0])
    y_acum.append(dejabot.q[1])
    x_dot_acum.append(dejabot.q_dot[0])
    u_hist.append(dejabot.get_u(dejabot, dt, envir, False))
    alpha_hist.append(calc_alpha(dejabot.q, goal))
x_acum = np.array(x_acum)
y_acum = np.array(y_acum)
x_dot_acum = np.array(x_dot_acum)
u_hist = np.array(u_hist)
alpha_hist = np.array(alpha_hist)
plt.plot(np.arange(n+1), x_acum, label = 'x')
plt.plot(np.arange(n+1), y_acum, label = 'y')
plt.plot(np.arange(n+1), x_dot_acum, label = 'x_dot')
plt.plot(np.arange(n+1), u_hist[:,0]*0.01, label = 'u_x')
plt.plot(np.arange(n+1), alpha_hist/np.pi, label = 'alpha')
plt.legend()
plt.grid()

In [ ]:
plt.plot(x_acum, y_acum, marker = '.', markersize = 3)

In [ ]:
def calc_field_value(q, envir):
    goal, obstacles = envir
    x, y, psi = q
    z = ((goal[0]-x)**2 + (goal[1]-y)**2)**0.5
    for obs in obstacles:
        dist = ((obs[0]-x)**2 + (obs[1]-y)**2)**0.5
        z = z + np.where(dist <= obs[2], 2, 1.5/((dist - obs[2] + 0.025)*40)**4)
    return z

In [ ]:
x = np.linspace(0,1,100)
y = x
xx,yy = np.meshgrid(x, y)
zz = calc_field_value([xx,yy,0], envir)

In [ ]:
plt.imshow(zz, origin = 'lower')
plt.colorbar()

In [ ]:
def first_der_grad_dir(q, ds, func, envir):
    x, y, psi = q
    dx = np.array([ds,0,0])
    dy = np.array([0,ds,0])
    dfx = func(q+dx, envir) - func(q, envir)
    dfy = func(q+dy, envir) - func(q, envir)
    #print (f'x:{x} dx: {dx} f(q){func(q, envir)} f(q+dx){func(q+dx, envir)} dfx: {dfx}')
    alpha = calc_alpha([0,0], [-dfx, -dfy])
    return alpha

In [ ]:
goal = [1,1,0]
obstacles = [
    [0.5, 0.3, 0.1],
    [0, 0.6, 0.02]
]
envir = [goal, obstacles]
def better_grad_desc(bot, dt, envir, print_m = False):
    q = bot.q
    goal, obstacles = envir
    q_dot = bot.q_dot
    next_q = q + q_dot*dt #not taking on account acceleration, unknown at this point
    
    alpha = first_der_grad_dir(next_q, dt*0.005, calc_field_value, envir)
    v_unit = np.array([np.cos(alpha), np.sin(alpha),0])
    max_speed = max_speed_dir(bot, alpha)
    dist = np.linalg.norm(np.matrix(goal)-next_q)
    
    one_step_speed = dist/dt
    speed = min(max_speed,  one_step_speed)
    delta_v = speed*v_unit - q_dot
    
    beta = calc_alpha([0,0,0], delta_v)
    max_acc = bot.max_q_sec_dir(beta, 0, False)
    one_step_accel = np.linalg.norm(delta_v)/dt
    accel = min(max_acc, one_step_accel)
    
    u = [accel*np.cos(beta), accel*np.sin(beta),0]
    if print_m:
        print(f'delta_v = {np.round(delta_v,4)}, beta = {np.round(beta*180/np.pi,4)}, a = {np.round(max_acc,4)} u = {np.round(u,4)}')
    
    return(np.array(u))

In [ ]:
goal = [1,1,0]
obstacles = [
    [0.6, 0.4, 0.15],
    [0, 0.6, 0.02],
    [0.7, 0.85, 0.08]
]
envir = [goal, obstacles]
dt = 0.1
n = 700
q_0 = np.array([0, 0, 0])
q_dot_0 = np.array([0, 0, 0])
dejabot = Bot(geom, mass, q_0, q_dot_0, better_grad_desc)
x_acum = [dejabot.q[0],]
x_dot_acum = [dejabot.q[0],]
y_acum = [dejabot.q[1],]
u_hist = [dejabot.get_u(dejabot, dt, envir)]
alpha_hist = [first_der_grad_dir(dejabot.q, dt*0.005, calc_field_value, envir)]
for i in range(n):
    dejabot.step(dt,envir)
    x_acum.append(dejabot.q[0])
    y_acum.append(dejabot.q[1])
    x_dot_acum.append(dejabot.q_dot[0])
    #print('\nq = ', np.round(dejabot.q, 4),'q_dot = ', np.round(dejabot.q_dot, 4))
    u_hist.append(dejabot.get_u(dejabot, dt, envir, False))
    alpha_hist.append(calc_alpha(dejabot.q, goal))
x_acum = np.array(x_acum)
y_acum = np.array(y_acum)
x_dot_acum = np.array(x_dot_acum)
u_hist = np.array(u_hist)
alpha_hist = np.array(alpha_hist)
plt.plot(np.arange(n+1), x_acum, label = 'x')
plt.plot(np.arange(n+1), y_acum, label = 'y')
#plt.plot(np.arange(n+1), x_dot_acum, label = 'x_dot')
#plt.plot(np.arange(n+1), u_hist[:,0]*0.01, label = 'u_x')
#plt.plot(np.arange(n+1), alpha_hist/np.pi, label = 'alpha')
plt.legend()
plt.grid()

In [ ]:
plt.figure(figsize = [12,10])
x = np.linspace(-0.1,1.1,100)
y = x
xx,yy = np.meshgrid(x, y)
zz = calc_field_value([xx,yy,0], envir)
plt.imshow(zz, extent=[-0.1,1.1,-0.1,1.1], origin = 'lower')
plt.plot(x_acum, y_acum,'r', marker = '.', markersize = 2)
plt.colorbar()

## Simulation stop criteria

In [ ]:
def stop_cond_pos(bot, envir, eps_q = 0.001, eps_q_dot = 0.1):
    q = bot.q
    goal, obstacles = envir
    q_dot = bot.q_dot
    
    c1 = np.linalg.norm(np.matrix(goal)-q)
    c2 = np.linalg.norm(q_dot)
    
    #print(f'c1 = {c1}, c2 = {c2}')
    if c1<eps_q and c2<eps_q_dot:
        return True
    else:
        return False    

In [ ]:
def simulation(bot, envir, dt = 0.01, n = 1000,
               q_0 = np.array([0, 0, 0]), q_dot_0 = np.array([0, 0, 0]),
               print_u = False, print_state = False):
    x_acum = [bot.q[0],]
    x_dot_acum = [bot.q[0],]
    y_acum = [bot.q[1],]
    u_hist = [bot.get_u(bot, dt, envir)]
    alpha_hist = [first_der_grad_dir(bot.q, dt*0.005, calc_field_value, envir)]
    for i in range(n):
        bot.step(dt,envir)
        x_acum.append(bot.q[0])
        y_acum.append(bot.q[1])
        x_dot_acum.append(bot.q_dot[0])
        if print_state:
            print('\nq = ', np.round(bot.q, 4),'q_dot = ', np.round(bot.q_dot, 4))
        u_hist.append(bot.get_u(bot, dt, envir, print_u))
        alpha_hist.append(calc_alpha(bot.q, goal))
        if stop_cond_pos(bot, envir):
            break
    x_acum = np.array(x_acum)
    y_acum = np.array(y_acum)
    x_dot_acum = np.array(x_dot_acum)
    u_hist = np.array(u_hist)
    alpha_hist = np.array(alpha_hist)
    
    return x_acum, y_acum, x_dot_acum, u_hist, alpha_hist

In [ ]:
goal = [1,1,0]
obstacles = [
    [0.6, 0.4, 0.15],
    [0, 0.6, 0.02],
    [0.7, 0.85, 0.08]
]
envir = [goal, obstacles]
dt = 0.1
n = 1000
q_0 = np.array([0, 0, 0])
q_dot_0 = np.array([0, 0, 0])
dejabot = Bot(geom, mass, q_0, q_dot_0, better_grad_desc)

results = simulation(dejabot, envir, dt, n, q_0, q_dot_0, print_u = False, print_state = False)
x_acum, y_acum, x_dot_acum, u_hist, alpha_hist = results

plt.plot(np.arange(len(x_acum)), x_acum, label = 'x')
plt.plot(np.arange(len(x_acum)), y_acum, label = 'y')
#plt.plot(np.arange(n+1), x_dot_acum, label = 'x_dot')
#plt.plot(np.arange(n+1), u_hist[:,0]*0.01, label = 'u_x')
#plt.plot(np.arange(n+1), alpha_hist/np.pi, label = 'alpha')
plt.legend()
plt.grid()

In [ ]:
plt.figure(figsize = [12,10])
x = np.linspace(-0.1,1.1,100)
y = x
xx,yy = np.meshgrid(x, y)
zz = calc_field_value([xx,yy,0], envir)
plt.imshow(zz, extent=[-0.1,1.1,-0.1,1.1], origin = 'lower')
plt.plot(x_acum, y_acum,'r', marker = '.', markersize = 2)
plt.colorbar()